<a href="https://colab.research.google.com/github/karthikv1392/eng-agenticai/blob/main/Agentic_AI_Tutorial_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to the Tutorial on Engineering Agentic AI Systems**

This the first part of the tutorial. It consists of the following set of activities that we will be going through:

1.   Building a simple reactive agent
2.   Building a RAG based agent
3.   Agent with tool calling and memory capability
4.   Simple orchestration flow of agents
5.   Multi-agent interactions



**Let first see the Reactive Agent part**

To use a generative model, you'll need an API key. If you don't already have one, create a key in Google AI Studio.
In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [1]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Before you can make any API calls, you need to initialize the Generative Model.

In [2]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

Now you can define a simple function that takes a prompt and returns the model's response.

In [3]:
def simple_reactive_agent(task):
  """
  Takes a task and returns the model's response.

  Args:
    task: The input task string.

  Returns:
    The response text from the generative model.
  """
  response = gemini_model.generate_content(task)
  return response.text

You can now use the `simple_prompt_agent` function. For example:

In [4]:
task = "Give me a plan to do a presentation on Agentic AI"
response = simple_reactive_agent(task)
print(response)

Okay, here's a plan to help you build a compelling presentation on Agentic AI, covering everything from content structure to delivery tips.

**I.  Understanding Your Audience & Defining Scope**

*   **A.  Audience Analysis:**
    *   **Technical Background:** Are they technical experts, business leaders, or a general audience?  This will determine the depth of technical detail.
    *   **Prior Knowledge:** What do they already know about AI, automation, and related concepts?
    *   **Interests/Motivations:** Why are they attending this presentation? What problems might they be trying to solve? Are they looking for investment opportunities, strategic insights, or just general knowledge?
    *   **Desired Takeaways:**  What key messages do you want them to remember?

*   **B.  Presentation Goal:**
    *   **Primary Objective:** What is the single, most important thing you want the audience to understand or do after the presentation? (e.g., "Understand the potential of Agentic AI to tran

Now that the simple reactive agent is built, can we ask the LLM to check if it can provide some specific response to some NASSCOM guidelines or report.

In [5]:
task = "According to the latest NASSCOM report on the future of AI in India, what are the key recommendations for startups in the healthcare sector?"
response = simple_reactive_agent(task)
print(response)

While I don't have access to a specific, timestamped "latest NASSCOM report on the future of AI in India," I can give you a synthesis of common recommendations for healthcare startups leveraging AI, based on numerous NASSCOM reports, industry discussions, and general best practices in the field.  These recommendations generally fall under the following key themes:

**1. Data Strategy and Management:**

*   **Focus on Data Quality and Accessibility:** NASSCOM often emphasizes the critical importance of high-quality, well-annotated, and accessible data.  Startups should prioritize building robust data pipelines, ensuring data accuracy, and adhering to ethical data collection and storage practices. They need to be able to access relevant datasets, which might require partnerships with hospitals, clinics, or research institutions.

*   **Data Security and Privacy Compliance (HIPAA/DPDP Act/ Equivalent):**  Data security and patient privacy are paramount. Startups *must* comply with all rel

First, let's make sure we have the necessary libraries installed.

In [6]:
%pip install PyMuPDF langchain langchain-community langchain-google-genai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires g

Now, we'll write the code to read the PDF, chunk the text, create embeddings, and build the FAISS index. Make sure your PDF file is uploaded to the Colab environment (you can drag and drop it into the "Files" tab on the left). We'll assume the PDF is in the `/content/` directory.

In [7]:
import fitz # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

import os
import faiss

# --- Configuration ---
PDF_FILENAME = "nasscom-zinnov-indian-tech-start-up-report-2023.pdf" # Replace with the actual name of your uploaded PDF
PDF_PATH = f"/content/{PDF_FILENAME}"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
#EMBEDDING_MODEL = "models/embedding-001"

# --- Check if the PDF file exists ---
if not os.path.exists(PDF_PATH):
    print(f"Error: PDF file not found at {PDF_PATH}")
else:
    # --- Read the PDF ---
    print(f"Reading PDF from {PDF_PATH}...")
    text = ""
    try:
        doc = fitz.open(PDF_PATH)
        for page in doc:
            text += page.get_text()
        print(f"Successfully read {len(text)} characters from {PDF_FILENAME}.")

        # --- Split the text into chunks ---
        print("Splitting text into chunks...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )
        docs = text_splitter.create_documents([text])
        print(f"Split the document into {len(docs)} chunks.")

        # --- Create embeddings and build FAISS index ---
        if docs:
            print("Creating embeddings and building FAISS index...")
            # Initialize Google Generative AI embeddings
            # Ensure GOOGLE_API_KEY is set in Colab secrets or environment variables
            #embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
            embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            # Create an in-memory FAISS vector store from the document chunks and embeddings
            vectorstore = FAISS.from_documents(docs, embeddings)
            print("Successfully created FAISS vector store.")

            # You can now use 'vectorstore' for similarity searches
            # Example:
            # query = "What are the key recommendations for startups in the healthcare sector?"
            # docs_and_scores = vectorstore.similarity_search_with_score(query)
            # for doc, score in docs_and_scores:
            #     print(f"Score: {score}")
            #     print(f"Content: {doc.page_content}\n")

        else:
            vectorstore = None
            print("No document chunks available to create vector store.")

    except Exception as e:
        print(f"An error occurred: {e}")

Reading PDF from /content/nasscom-zinnov-indian-tech-start-up-report-2023.pdf...
Successfully read 76946 characters from nasscom-zinnov-indian-tech-start-up-report-2023.pdf.
Splitting text into chunks...
Split the document into 97 chunks.
Creating embeddings and building FAISS index...


/tmp/ipython-input-178021777.py:44: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Successfully created FAISS vector store.


**Building a simple RAG Agent**

In [8]:
from langchain.chains import RetrievalQA

from langchain_google_genai import ChatGoogleGenerativeAI


def simple_rag_agent (task):
  """
  Takes a task, checks how the task can be accomplished using the knowledge
  and returns the model's response.

  Args:
    task: The input task string.

  Returns:
    The response text by augumenting the knowledge that is there by the Generative model.
  """
  llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=userdata.get('GOOGLE_API_KEY')
    )
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
  )
  response = qa_chain.run(task)
  return response


In [9]:
task_rag = "what are the key recommendations for startups from NASSCOM?"
action_rag = simple_rag_agent(task_rag)
print (action_rag)

/tmp/ipython-input-3154212237.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(task)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Based on the provided text, the document describes what tech start-up founders prioritize and are seeking, but it does not explicitly list "key recommendations for startups from NASSCOM."

Instead, it highlights that:
*   **Market access** is reported as a key factor for success by tech start-up founders.
*   **Collaboration with government and corporates** is being sought by the majority of tech start-ups (56% of funded, 60% of unfunded seeking corporate collaboration, and 70% of unfunded seeking government collaboration).
*   **Funding** is the second most important factor for unfunded start-ups.
*   **Talent** is the second most important factor for funded start-ups.


**Does the agent has memory? What about invoking tools?**

In [10]:
task_rag = "Based on the above what do you suggest for me as a startup?"
action_rag = simple_rag_agent(task_rag)
print (action_rag)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Based on the provided text, here are some suggestions for a startup:

1.  **Focus on Strong Business Fundamentals:** The ecosystem is shifting towards profitability and revenue generation. Over 60% of tech start-up founders reported an increase in revenue and profitability in 2023, and over 60% expect to increase revenues next year. This suggests a need for a sustainable business model from the outset.

2.  **Consider DeepTech, especially AI/Generative AI:**
    *   New startups are increasingly focusing on DeepTech (25% in 2023 vs. ~12% in the last 2 years).
    *   Over 100+ Generative AI startups are active in India.
    *   70% of startup founders intend to embed AI in their solutions.
    *   Investments in DeepTech are likely to continue.

3.  **Explore Opportunities in Key Growth Sectors:** The text highlights a surge in several sectors over the past decade and their continued growth:
    *   **EnterpriseTech:** Continues to be a significant and growing sector.
    *   **HealthT

**Any Observations?**

The agent didn't have the previous chat context to answer in a more detailed manner. In other words, the agent does not have any memory. What if we want the agent to answer question like what is the day today. Even that will fail!! There is a lack of context. What does "today" mean. LLM does not know it

In [11]:
task_rag = "What is the latest news as of today?"
action_rag = simple_rag_agent(task_rag)
print (action_rag)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


I cannot provide the latest news as of today. The provided text is a report that contains data and outlooks up to December 2023 and some projections for 2024. It does not contain real-time news updates.


In [12]:
# What if we had asked this question to our old simple agent
action_simple = simple_reactive_agent(task_rag)
print (action_simple)

Okay, here's a quick rundown of some of the top news stories as of today, October 26, 2023.  Please keep in mind that news is constantly evolving, so this is just a snapshot:

*   **Middle East Conflict:** The conflict between Israel and Hamas continues to be a major focus. Efforts to negotiate a humanitarian pause to allow aid into Gaza are ongoing, but remain challenging. International pressure is mounting for both sides to protect civilians. There are ongoing concerns about the potential for the conflict to escalate into a wider regional war.
*   **Economic News:** Economic data releases are being closely watched for signals about the strength of the global economy and the potential for recession. Interest rate decisions by central banks are also in focus.
*   **Political Developments:** Various political events are unfolding around the world, including elections, government formations, and policy debates.
*   **Other Notable Events:** Other events such as major weather events, scie

**Quick Observations and What Next**

1. The date given is as per the knowledge of the LLM used.
2. Need ways to invoke APIs
3. We also need ways to store context of previous conversation

Two more capabilities - Memory and tool calling

In [13]:
## Lets build some functions for tooling

from datetime import datetime

def get_current_date():
    return f"Today's date is {datetime.now().strftime('%Y-%m-%d')}."

def add_numbers(numbers_str):
    try:
        numbers = list(map(float, numbers_str.split()))
        return f"The sum is {sum(numbers)}."
    except:
        return "Please provide numbers separated by spaces."

def greet(name):
    return f"Hello, {name}! Nice to meet you."

In [14]:
## Adding simple memory - Just conversation history

chat_history = []

def simple_tool_augumented_agent(task):
  """
  Takes a task, checks how the task can be accomplished using the tools and memory
  and returns the model's response.

  Args:
    task: The input task string.

  Returns:
    The response text by ysing the tools with the support of the generative model.
  """
  history_prompt = "\n".join([f"User: {u}\nAI: {a}" for u, a in chat_history])
  prompt = f"""
    You are a helpful assistant that can use tools.
    Available tools:
    1. get_current_date() - returns the current date.
    2. add_numbers("number1 number2 ...") - returns the sum.
    3. greet("name") - greets the user.

    Use the tools if needed. Otherwise, answer directly by checking the history of conversation

    {history_prompt}
    User: {task}
    What would you like to do? If a tool is needed, return a Python function call in this format: tool_call: TOOL_FUNCTION_CALL_HERE
  """
  response = gemini_model.generate_content(prompt)
  reply = response.text.strip()
  ## Let us check what was the repsonse from the Generative AI model based on the instructions
  print (reply)
  full_reply = ""
  if "tool_call:" in reply:
    tool_call = reply[len("tool_call: "):].strip()
    try:
      print (tool_call)
      result = eval(tool_call)
      full_reply = result
      chat_history.append((task, full_reply))
      return full_reply
    except Exception as e:
      full_reply = f"{reply}\n\n[Tool Error]: {e}"
    else:
        full_reply = reply

  chat_history.append((task, full_reply))
  print (chat_history)
  return full_reply


In [15]:
response = simple_tool_augumented_agent("What is the date today?")
print (response)

tool_call: get_current_date()
get_current_date()
Today's date is 2025-08-06.


In [16]:
response = simple_tool_augumented_agent("What did we just talk about?")
print (response)

We just talked about the date, which is 2025-08-06.
[('What is the date today?', "Today's date is 2025-08-06."), ('What did we just talk about?', '')]



In [17]:
response = simple_tool_augumented_agent("Can you greet me? My name is Raj")
print (response)

```tool_call
greet(name='Raj')
```
[('What is the date today?', "Today's date is 2025-08-06."), ('What did we just talk about?', ''), ('Can you greet me? My name is Raj', '')]



In [18]:
response = simple_tool_augumented_agent("What is my name?")
print (response)

Your name is Raj.
[('What is the date today?', "Today's date is 2025-08-06."), ('What did we just talk about?', ''), ('Can you greet me? My name is Raj', ''), ('What is my name?', '')]



Now lets move to Type 4 agentic AI where the agents try to communicate to other with the help if

**Moving to Type 4: Orchestration**

Consider a scenario where you would like different agents to collaborate but the order of execution is clear and the set of agents required to achieve the use case is also clear. In such a scenario, we can build an orchestrated agentic AI system where the interactions between the agents are orchestrated as in the case of an orchestration in microservices.

Please note that the error can still occur if an agent does not perform the task as it is supposed to perform. Ultimately there is lot of uncertainity involved. In this type the only advantage is that the flow is defined.

**City Travel Planner System**

The goal is to build a city travel planner system where given a city and some budget, the system will suggest the most optimal plan.

The following are the tools that is being made available and the set of agents that will be in use.

In [19]:
def budget_calculator(plan_text):
    """
    Estimate budget based on types of activities.
    For simplicity, we'll assign:
      - Museum Visit: ₹300
      - Lunch/Dinner: ₹500
      - Local Travel: ₹200
      - Shopping: ₹1000
      - Park/Nature: ₹100
    """
    budget = 0
    plan_text = plan_text.lower()
    if "museum" in plan_text:
        budget += 300
    if "lunch" in plan_text or "dinner" in plan_text or "food" in plan_text:
        budget += 500
    if "travel" in plan_text or "transport" in plan_text:
        budget += 200
    if "shopping" in plan_text:
        budget += 1000
    if "park" in plan_text or "garden" in plan_text or "nature" in plan_text:
        budget += 100
    return f"Estimated budget for the day is ₹{budget}."

In [20]:
def get_weather(city):
    return f"The weather in {city} is sunny with a high of 30°C."

In [21]:
def planner_agent(city):
    prompt = f"Plan a one-day trip to {city}. Include 3-4 diverse activities with morning to evening flow."
    return gemini_model.generate_content(prompt).text

In [22]:
def budget_agent(plan_text):
    tool_output = budget_calculator(plan_text)
    prompt = f"""Here is the initial plan:
{plan_text}

The budget estimation tool says:
{tool_output}

Now revise the plan if needed to make it cost-effective (under ₹1500), and list the final activities."""
    return gemini_model.generate_content(prompt).text, tool_output


In [23]:
def reviewer_agent(revised_plan, city):
    weather = get_weather(city)
    prompt = f"""Here is the revised plan:
{revised_plan}

Today's weather in {city} is:
{weather}

Refine the plan further if the weather affects any activity. Make sure the plan is practical."""
    return gemini_model.generate_content(prompt).text, weather

In [24]:
def plan_generator_agent(refined_plan, budget_info, weather_info):
    prompt = f"""You are a friendly itinerary assistant.

Given:
Plan:
{refined_plan}

Weather:
{weather_info}

Budget Info:
{budget_info}

Generate a final itinerary for the day from morning to evening with friendly timings and formatting."""
    return gemini_model.generate_content(prompt).text

In [25]:
def orchestrate_city_trip(city):
    print("📍 Starting trip planning for:", city)

    initial_plan = planner_agent(city)
    print("\n🧠 [Planner Output]\n", initial_plan)

    revised_plan, budget_info = budget_agent(initial_plan)
    print("\n💰 [Budget Agent Output]\n", revised_plan)
    print("\n💸 [Budget Info]\n", budget_info)

    refined_plan, weather_info = reviewer_agent(revised_plan, city)
    print("\n🔍 [Reviewer Output]\n", refined_plan)
    print("\n🌤️ [Weather Info]\n", weather_info)

    final_itinerary = plan_generator_agent(refined_plan, budget_info, weather_info)
    print("\n📋 [Final Itinerary]\n", final_itinerary)

    return final_itinerary

In [26]:
orchestrate_city_trip("Rome")

📍 Starting trip planning for: Rome

🧠 [Planner Output]
 Okay, here's a plan for a one-day whirlwind tour of Rome, focusing on diverse experiences, from morning to evening.  It's ambitious, but doable if you're prepared for a fast-paced day and utilize public transport (Metro, buses) effectively.

**Theme:** Ancient Wonders & Roman Delights

**Morning (Ancient Glory):**

*   **8:00 AM - 11:00 AM: The Colosseum & Roman Forum/Palatine Hill**
    *   **Activity:**  Start your day early to beat the crowds at the Colosseum. Book tickets online in advance to skip the ticket line (essential!).
    *   **How:** Take the Metro Line B to "Colosseo" station.
    *   **Details:**  Explore the Colosseum (allow 1.5 hours). Then, walk over to the Roman Forum and Palatine Hill (allow 1.5 hours).  These were the heart of ancient Rome - imagine senators, emperors, and everyday life happening here.  Wear comfortable shoes, as there's a lot of walking on uneven surfaces.
    *   **Food:** Grab a quick coff

'Okay, here\'s your finalized, friendly-formatted itinerary for a fantastic, budget-friendly, and heat-beating day in Rome! Let\'s make those ancient stones sing!\n\n**Your Roman Adventure: Ancient Glory on a Budget (Beat the Heat Edition!)**\n\n**Theme:** Exploring Ancient Rome without breaking the bank (or melting in the sun!).\n\n**Key Weather Focus:** Hydration, shade, and sensible timing.\n\n**Budget:** ₹1000\n\n**Itinerary:**\n\n**Morning: Ancient Glory Awakens (FREE!)**\n\n*   **7:00 AM - 7:30 AM: Fuel Up Like a Roman!**\n    *   Grab an espresso and a delicious pastry at a local bar near your accommodation. Get that energy boost before the sun (and the crowds) really get going.\n    *   *Budget: ₹150*\n*   **7:30 AM - 10:00 AM: Colosseum & Forum Glimpses**\n    *   **Metro B to "Colosseo".**  Hop on the metro and head to the iconic Colosseum.\n    *   **Colosseum Exterior (30 mins):** Marvel at the Colosseum\'s grandeur from the outside. Take photos and soak it all in!  Remembe

Now, let us build the next one, which is **Type 5: Adaptive Multi-agent**. There is no notion of a central orchestrator here

Let us take the same use case of planning a trip to Rome.

All agents will implement a handle(input: dict) -> dict style method. Each agent receives a message dictionary and returns an enriched one.


In [45]:
# Shared blackboard and mailbox
blackboard = {}
mailbox = []

# Tool functions
def get_weather(city):
    return f"The weather in {city} is sunny with a high of 30°C."

def budget_calculator(plan_text):
    budget = 0
    plan_text = plan_text.lower()
    if "museum" in plan_text: budget += 300
    if any(w in plan_text for w in ["lunch", "dinner", "food"]): budget += 500
    if "travel" in plan_text or "transport" in plan_text: budget += 200
    if "shopping" in plan_text: budget += 1000
    if any(w in plan_text for w in ["park", "garden", "nature"]): budget += 100
    return f"Estimated budget for the day is ₹{budget}."


In [46]:
## creating a base class for Agents

class Agent:
    def __init__(self, name):
        self.name = name

    def send_message(self, goal, content, data={}):
        mailbox.append({
            "sender": self.name,
            "goal": goal,
            "content": content,
            "data": data
        })

    def can_handle(self, blackboard):
        raise NotImplementedError()

    def handle(self, blackboard):
        raise NotImplementedError()


In [47]:
class PlannerAgent(Agent):
    def can_handle(self, bb):
        return "city" in bb and "plan" not in bb

    def handle(self, bb):
        city = bb["city"]
        prompt = f"Plan a one-day trip to {city} with 4-5 diverse activities."
        plan = gemini_model.generate_content(prompt).text
        bb["plan"] = plan
        self.send_message("trip_planning", "Plan generated", {"plan": plan})


In [48]:
class BudgetAgent(Agent):
    def can_handle(self, bb):
        return "plan" in bb and "budget" not in bb

    def handle(self, bb):
        budget = budget_calculator(bb["plan"])
        prompt = f"""This is the plan:\n{bb['plan']}\n\nBudget tool says:\n{budget}\nRevise plan if over ₹1500."""
        revised = gemini_model.generate_content(prompt).text
        bb["budget"] = budget
        bb["revised_plan"] = revised
        self.send_message("trip_planning", "Budget handled", {"budget": budget})


In [49]:
class WeatherAgent(Agent):
    def can_handle(self, bb):
        return "revised_plan" in bb and "weather" not in bb

    def handle(self, bb):
        city = bb["city"]
        weather = get_weather(city)
        prompt = f"""Plan: {bb['revised_plan']}\n\nWeather in {city}: {weather}\nRefine if any issues."""
        final = gemini_model.generate_content(prompt).text
        bb["weather"] = weather
        bb["final_plan"] = final
        self.send_message("trip_planning", "Weather refined", {"weather": weather})


In [50]:
class PlanGeneratorAgent(Agent):
    def can_handle(self, bb):
        return "final_plan" in bb and "itinerary" not in bb

    def handle(self, bb):
        prompt = f"""Create a final itinerary:\n\nPlan: {bb['final_plan']}\n\nWeather: {bb['weather']}\nBudget: {bb['budget']}"""
        itinerary = gemini_model.generate_content(prompt).text
        bb["itinerary"] = itinerary
        self.send_message("trip_planning", "Final itinerary generated", {"itinerary": itinerary})
        print("✅ Final Itinerary:\n")
        print(itinerary)


In [51]:
# Register agents
agents = [
    PlannerAgent("Planner"),
    BudgetAgent("Budgeter"),
    WeatherAgent("Weatherer"),
    PlanGeneratorAgent("Generator")
]

# Kick off the system
blackboard["city"] = "Rome"
mailbox.append({"sender": "User", "goal": "trip_planning", "content": "Start trip planning", "data": {}})

# Main loop
def run_system():
    while any(agent.can_handle(blackboard) for agent in agents):
        for agent in agents:
            if agent.can_handle(blackboard):
                agent.handle(blackboard)

In [52]:
run_system()

✅ Final Itinerary:

Okay, here is the final, finely-tuned itinerary for a day in Rome on a budget of ₹2100, incorporating all your excellent refinements.  This plan prioritizes seeing key sights, experiencing local food, and staying within budget.

**Rome on a Shoestring: A One-Day Itinerary (₹2100 Budget)**

**Morning (8:00 AM - 12:00 PM): Ancient Rome & Trastevere Charm**

*   **8:00 AM:** Start at the **Colosseum** (External View - Free).  Take photos and admire the iconic structure from the outside.  Consider researching its history beforehand to enhance the experience. *Reasoning: Entry fee is too high for the budget. Focus on the external grandeur.*
*   **8:30 AM:** Walk towards the **Roman Forum** and **Palatine Hill** (External View - Free).  Again, enjoy the views from outside the ticketed areas. Imagine the bustling life of ancient Rome. *Reasoning: As above, prioritize free viewpoints.*
*   **9:30 AM:** Head towards **Trastevere**. Enjoy the charming atmosphere of this neigh